In [187]:
import pandas as pd
import yaml
import requests
import json

In [188]:
secrets_location = "C:\\Users\\lowsona\\Github\\Supporting Files\\secrets.yml"
secrets = open(secrets_location)
secrets = yaml.load(secrets, Loader=yaml.FullLoader)['SharePoint_LGBF']
SP_clientsecret = secrets['client_secret']
SP_tenantid = secrets['tenant_id']
SP_tenant = "stirlingcounciluk"
SP_clientid = secrets['client_id'] + '@' + SP_tenantid

In [189]:
# Establish data to be passed as part of POST request to obtain session token from SharePoint
data = {
    'grant_type': 'client_credentials',
    'resource': "00000003-0000-0ff1-ce00-000000000000/" + SP_tenant + ".sharepoint.com@" + SP_tenantid,
    'client_id': SP_clientid,
    'client_secret': SP_clientsecret,
}

# Establish headers to be passed as part of POST request to obtain session token from SharePoint
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Construct url variable for token request
url = "https://accounts.accesscontrol.windows.net/" + \
    SP_tenantid + "/tokens/OAuth/2"

# Send POST request to obtain session token and then load the response into json_data
r = requests.post(url, data=data, headers=headers)
json_data = json.loads(r.text)

In [190]:
# Establish headers to be passed as part of GET request to obtain data from SharePoint list. Using the 'access_token' obtained in the previous step
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept': 'application/json;odata=verbose',
    "Accept-Charset": "utf-8",
    'Content-Type': 'application/json;odata=verbose'
}

# Construct url variable for data request
url = "https://" + SP_tenant + \
    ".sharepoint.com/sites/PPA/LGBF/_api/web/lists/GetByTitle('Indicator%20Information')/items"

Items = []

# Send GET request to obtain list data. Extract and normalize this data into a dataframe.


def getItems(url):
    Itemsrequest = requests.get(url, headers=headers)
    Itemsjson = Itemsrequest.json()
    Item = pd.json_normalize(Itemsjson['d']['results'])
    Items.append(Item)
    if '__next' in Itemsjson['d']:
        newurl = Itemsjson['d']['__next']
        getItems(newurl)


getItems(url)

indicatorInformation = pd.concat(Items)

indicatorInformation.head(3)

,FileSystemObjectType,Id,ServerRedirectedEmbedUri,ServerRedirectedEmbedUrl,Title,Code,Code_x0028_SortableA_x002d_Z_x00,ReportingPeriod,MeasureType,NumberFormat,...,GetDlpPolicyTip.__deferred.uri,FieldValuesAsHtml.__deferred.uri,FieldValuesAsText.__deferred.uri,FieldValuesForEdit.__deferred.uri,File.__deferred.uri,Folder.__deferred.uri,LikedByInformation.__deferred.uri,ParentList.__deferred.uri,Properties.__deferred.uri,Versions.__deferred.uri
0,0,18,None,,Net Cost of Waste Collection per Premises,ENV01a,ENV 01a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
1,0,19,None,,Net Cost of Waste Disposal per Premises,ENV02a,ENV 02a,Annual,Cost,'£ '0,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...
2,0,20,None,,The % of Total Household Waste Arising that is...,ENV06,ENV 06,Annual,Percentage,0.0%,...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...,https://stirlingcounciluk.sharepoint.com/sites...


In [191]:
# Drop system columns not needed in dataframe
indicatorInformation = indicatorInformation.drop(columns=['FileSystemObjectType', 'Id', 'ServerRedirectedEmbedUri', 'ServerRedirectedEmbedUrl', 'GetDlpPolicyTip.__deferred.uri', 'FieldValuesAsHtml.__deferred.uri', 'FieldValuesAsText.__deferred.uri', 'FieldValuesForEdit.__deferred.uri', 'File.__deferred.uri', 'Folder.__deferred.uri', 'LikedByInformation.__deferred.uri', 'ParentList.__deferred.uri', 'Properties.__deferred.uri',
                                                 'Versions.__deferred.uri', 'Attachments', 'GUID', '__metadata.id', '__metadata.uri', '__metadata.etag', '__metadata.type', 'FirstUniqueAncestorSecurableObject.__deferred.uri', 'RoleAssignments.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'AttachmentFiles.__deferred.uri', 'ContentType.__deferred.uri', 'ID', 'Modified', 'Created', 'AuthorId', 'EditorId', 'OData__UIVersionString', 'ContentTypeId', 'ComplianceAssetId'])

# Rename columns appropriately
indicatorInformation = indicatorInformation.rename(columns={'Code_x0028_SortableA_x002d_Z_x00': 'Code_Sortable', 'Ranking_x0020_Type': 'Ranking_Type',
                                                   'NumberFormat_x0028_notext_x0029_': "NumberFormat_NoText", "GoldilocksRankingMidpoint_x0028_": "Ranking_GoldilocksMidpoint"})

# Convert column types to ensure that they can be compared to other data later.
indicatorInformation['Code'] = indicatorInformation['Code'].astype(str)
indicatorInformation['Numerator_Match'] = indicatorInformation['Numerator_Match'].astype(str)
indicatorInformation['Denominator_Match'] = indicatorInformation['Denominator_Match'].astype(str)
#Convert number format to avoid 0.0% being parsed as a number and returning 0 when written to csv
indicatorInformation['NumberFormat'] = indicatorInformation['NumberFormat'].astype(str)

indicatorInformation.head(3)

,Title,Code,Code_Sortable,ReportingPeriod,MeasureType,NumberFormat,YMin,YMax,ISCategory,Committee,...,AdditionalAxisDenominator_Python,FormatAxis_Plotly_Prefix,FormatAxis_Plotly_Suffix,ImgPxlWidth_Plotly,ImgPxlHeight_Plotly,SubGroup_PythonReport,YMin_Plotly,YMax_Plotly,OData__ColorTag,Group_PythonReport
0,Net Cost of Waste Collection per Premises,ENV01a,ENV 01a,Annual,Cost,'£ '0,40.0,NaN,Environmental Services,Environment and Housing,...,1,£,None,800,493,Waste & Recycling,NaN,NaN,None,Environment
1,Net Cost of Waste Disposal per Premises,ENV02a,ENV 02a,Annual,Cost,'£ '0,60.0,NaN,Environmental Services,Environment and Housing,...,1,£,None,800,493,Waste & Recycling,70.0,120.0,None,Environment
2,The % of Total Household Waste Arising that is...,ENV06,ENV 06,Annual,Percentage,0.0%,40.0,100.0,Environmental Services,Environment and Housing,...,1,None,None,800,493,Waste & Recycling,0.4,0.6,None,Environment


In [192]:
indicatorInformation.to_csv(
    "Data Files//Indicator Information.csv", index=False, encoding='utf-8-sig')
